# IR Lab SoSe 2024: Baseline Retrieval System

This jupyter notebook serves as baseline retrieval system that you can try to improve upon.
We will use the a corpus of scientific papers (title + abstracts) from the fields of information retrieval and natural language processing (the [IR Anthology](https://ir.webis.de/anthology/) and the [ACL Anthology](https://aclanthology.org/)). This serves Jupyter notebook only serves as retrieval system, i.e., it gets a set of information needs (topics) and a corpus as input and produces a run file as output. Please do evaluations in a new dedicated notebook.

### Step 1: Import Libraries

We will use [tira](https://www.tira.io/), an information retrieval shared task platform, for loading the (pre-built) retrieval index and [ir_dataset](https://ir-datasets.com/) to subsequently build a retrieval system with [PyTerrier](https://github.com/terrier-org/pyterrier), an open-source search engine.

Building your own index can be already one way that you can try to improve upon this baseline (if you want to focus on creating good document representations). Other ways could include reformulating queries or tuning parameters or building better retrieval pipelines.

In [98]:
# You only need to execute this cell if you are using Google Golab.
# If you use GitHub Codespaces, everything is already installed.
!pip3 install tira ir-datasets python-terrier

In [99]:
# Imports
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
import pandas as pd

In [100]:
# Create a REST client to the TIRA platform for retrieving the pre-indexed data.
ensure_pyterrier_is_loaded()
tira = Client()

### Step 2: Load the Dataset and the Index

The type of the index object that we load is `<class 'jnius.reflect.org.terrier.structures.Index'>`, in fact a [Java class](http://terrier.org/docs/v3.6/javadoc/org/terrier/structures/Index.html) wrapped into Python. However, you do not need to worry about this: at this point, we will simply use the provided Index object to run procedures defined in Python.

In [101]:
# The dataset: the union of the IR Anthology and the ACL Anthology
# This line creates an IRDSDataset object and registers it under the name provided as an argument.
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

# A (pre-built) PyTerrier index loaded from TIRA
index = tira.pt.index('ir-lab-sose-2024/tira-ir-starter/Index (tira-ir-starter-pyterrier)', pt_dataset)

### Step 3: Define the Retrieval Pipeline

We will define a BM25 retrieval pipeline as baseline. For details, see:

- [https://pyterrier.readthedocs.io](https://pyterrier.readthedocs.io)
- [https://github.com/terrier-org/ecir2021tutorial](https://github.com/terrier-org/ecir2021tutorial)

In [102]:
#base retrieval model for candidate set with bm25
#additional models tf and pl2 serve as re-rankers in pipeline
#pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=["WMODEL:BM25Plus", "WMODEL:PL2", "WMODEL:DFRee", "WMODEL:Hiemstra_LM", "WMODEL:InL2", "WMODEL:LGD", "WMODEL:TF_IDF"])
pipeline = pt.FeaturesBatchRetrieve(index, wmodel="BM25", features=[
         "WMODEL:PL2", "WMODEL:DFRee", 
        "WMODEL:Hiemstra_LM", "WMODEL:InL2", "WMODEL:LGD", "WMODEL:TF_IDF"
    ])
#bm25 = pt.BatchRetrieve(index, wmodel="BM25")
#tf = pt.BatchRetrieve(index, wmodel="Tf")
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
#pipeline = bm25 >> (tf ** pl2)
#pipeline = (2* bm25 + tf + pl2).transform(topics)

### Step 4: Create the Run


In [103]:
print('First, we have a short look at the first three topics:')

pt_dataset.get_topics('text').head(3)

First, we have a short look at the first three topics:


,qid,query
0,1,retrieval system improving effectiveness
1,2,machine learning language identification
2,3,social media detect self harm


In [104]:
print('Now we do the retrieval...')
#run = (2* bm25 + tf + pl2).transform(pt_dataset.get_topics('text'))

#define empty feature dictionary
feature_dict = {}

#initial retrieve through pipeline
run = pipeline.transform(pt_dataset.get_topics('text'))
run = pipeline(pt_dataset.get_topics('text'))

# Convert the numpy arrays to lists
run['features'] = run['features'].apply(lambda x: x.tolist())

# Create new columns for each feature
#run['BM25Plus'] = run['features'].apply(lambda x: x[0])  # Assuming TF_IDF is the first feature
run['PL2'] = run['features'].apply(lambda x: x[0])
run['DFRee'] = run['features'].apply(lambda x: x[1])
run['Hiemstra_LM'] = run['features'].apply(lambda x: x[2])
run['InL2'] = run['features'].apply(lambda x: x[3])
run['LGD'] = run['features'].apply(lambda x: x[4])
run['TF_IDF'] = run['features'].apply(lambda x: x[5])

# compute reranking
reranked_run = run.sort_values(by=["PL2", "DFRee", "Hiemstra_LM", "InL2", "LGD", "TF_IDF"], ascending=[True, True, True, True, True, True])
reranked_run.reset_index(drop=True, inplace=True)


print('Done. Here are the first 10 entries of the run')
#run.head(10)
reranked_run.head(10)

Now we do the retrieval...
Done. Here are the first 10 entries of the run


,qid,query,docid,rank,features,docno,score,PL2,DFRee,Hiemstra_LM,InL2,LGD,TF_IDF
0,55,bm25,94720,0,"[-1.0816416702168659, 5.245637533519068, 1.128...",2009.cikm_conference-2009.247,15.882512,-1.081642,5.245638,1.128751,0.266463,3.918538,0.249012
1,55,bm25,79623,2,"[0.32153404451368217, 7.339907227116406, 2.615...",2011.sigirconf_conference-2011.121,14.919702,0.321534,7.339907,2.615362,1.014958,5.905596,0.976859
2,55,bm25,87286,1,"[0.9384013106425643, 8.031085395086288, 3.2279...",2019.fire_conference-2019w.7,15.231518,0.938401,8.031085,3.227989,1.510160,6.564471,1.482115
3,55,bm25,78722,4,"[1.2527507772648274, 8.341664583128031, 3.5179...",2015.iir_workshop-2015.14,14.455476,1.252751,8.341665,3.517997,1.785342,6.858476,1.771234
4,55,bm25,78816,5,"[1.348938519771023, 8.432047997059039, 3.60400...",2013.iir_workshop-2013.1,14.455476,1.348939,8.432048,3.604009,1.871690,6.943710,1.863193
5,45,analysis for android apps,106158,276,"[1.6628486110875271, 7.889250660566921, 3.1323...",2013.wwwconf_conference-2013c.166,6.282339,1.662849,7.889251,3.132343,2.441339,7.471012,2.483856
6,45,analysis for android apps,67442,582,"[1.733785119954963, 7.9549676953579445, 3.1942...",2021.paclic-1.55,4.673411,1.733785,7.954968,3.194270,2.516932,7.531778,2.568202
7,45,analysis for android apps,85310,890,"[1.748482803503374, 7.968473913729951, 3.20706...",2000.riao_conference-2000.22,4.545224,1.748483,7.968474,3.207064,2.532621,7.544252,2.585763
8,55,bm25,91659,3,"[1.7617220723878595, 8.79885007209927, 3.96051...",2020.ecir_conference-20202.4,14.570957,1.761722,8.798850,3.960513,2.250401,7.287876,2.273510
9,45,analysis for android apps,53195,368,"[1.8326063594697175, 8.045077665575892, 3.2800...",D14-1221,5.036380,1.832606,8.045078,3.280060,2.622565,7.614905,2.686812


### Step 5: Persist the run file for subsequent evaluations

The output of a prototypical retrieval system is a run file. This run file can later (optimally in a different notebook) be statistically evaluated.

In [105]:
persist_and_normalize_run(run, system_name='bm25-baseline', default_output='../runs')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs".
Done. run file is stored under "../runs/run.txt".
